In [15]:
#Import packages
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import docx
from word_tables import output_to_table
import warnings # ignore warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [16]:
food_data = pd.read_csv("../data/data_inprogress.csv", index_col=0)
food_data.reset_index(inplace=True)
print(food_data.shape)
food_data.head()

(69, 29)


,index,item_name,price,menu_section,Calories,Total Fat,Saturated Fat,Trans Fat,Cholesterol,Sodium,Total Carbohydrates,Dietary Fiber,Protein,Vitamin D,Calcium,Iron,Potassium,Gluten,Milk,Wheat,Soy,Eggs,Fish,Shellfish,Tree Nuts,Peanuts,MSG,Sesame,Total Sugars
0,0,Soft Taco,1.89,Tacos,180.0,8.0,4.0,0.0,25.0,500.0,18.0,3.0,9.0,0.0,110.0,1.7,130.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,1,Soft Taco Supreme®,2.89,Tacos,210.0,10.0,5.0,0.0,25.0,510.0,20.0,3.0,10.0,0.0,130.0,1.7,200.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,2,Spicy Potato Soft Taco,1.10,Tacos,240.0,12.0,3.0,0.0,10.0,480.0,28.0,2.0,5.0,0.0,110.0,1.3,270.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,Crunchy Taco,1.89,Tacos,170.0,10.0,3.5,0.0,25.0,300.0,13.0,3.0,8.0,0.0,70.0,0.9,140.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,Crunchy Taco Supreme®,2.89,Tacos,190.0,11.0,4.5,0.0,25.0,320.0,15.0,3.0,8.0,0.0,80.0,0.9,200.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


## RQ: Can Taco Bell's menu provide a balanced diet that aligns with affordability and nutritional requirements, especially for individuals with limited financial means or access to diverse food sources?

### Sets and Indices
Let $F$ denote the set of food items considered on the menu: $i \in F$

Let $N$ denote the set of nutritional requirements: $j \in N$.

### Data
$n_{i,j}$ = How much of nutrient $j$ one menu item $i$ has, $i \in F, j \in N$.

$c_i$ = The price of one serving of menu item $i$, $i \in F$ (per item cost)

$m_j$ = Minimum daily requirements of nutrient $j$, $j
\in N$


## Model 1: Minimizing costs
### Decision Variable
* $x_i$: The number of menu items $i$ to include in the diet; $x_1,..,n$


### Objective
* Minimize costs $\sum_{i \in F} c_i * x_i$.


### Constraints
* $ \sum_{i \in F} n_{i,j} * x_i \geq m_j,  j \in N$
* $x_i \geq 0, i=1,\ldots,n$ (non-negativity constraint)

- All nutitonal goals can be found here: https://www.dietaryguidelines.gov/sites/default/files/2020-12/Dietary_Guidelines_for_Americans_2020-2025.pdf


##### The calculations to find out the range of total fat and saturated fat allowed in a 2400 calorie-a-day-diet
- Based on the diet for Males ages 19-30, the 2015-2020 Dietary Guidelines for Americans recommend the following:
    - Total fat: 20% to 35% of daily calories
    - Saturated fat: 10% or less of daily calories
- There are 9 calories in a gram of fat, so we will divide the number of calories by 9. 

source: https://newsnetwork.mayoclinic.org/discussion/fat-grams-how-to-track-fat-in-your-diet/#:~:text=The%202015%2D2020%20Dietary%20Guidelines,to%2035%25%20of%20daily%20calories

In [17]:
min_totFat = (2400*0.2)//9
max_totFat = (2400*0.35)//9
max_statFat = (2400*0.10)//9

print(min_totFat)
print(max_totFat)
print(max_statFat)

53.0
93.0
26.0


In [18]:
#Defining variables
price = food_data['price'].tolist()
menu_item = food_data['item_name'].tolist()
section_name = food_data['menu_section'].to_list()

calories = food_data['Calories'].tolist()
protein = food_data['Protein'].tolist()
totalCarbohydratets = food_data['Total Carbohydrates'].tolist()
dietaryFiber = food_data['Dietary Fiber'].tolist()
totfat = food_data['Total Fat'].tolist()
statFat = food_data['Saturated Fat'].to_list()
transFat = food_data['Trans Fat'].to_list()
cholesterol = food_data['Cholesterol'].tolist()
sodium = food_data['Sodium'].tolist()
sugars = food_data['Total Sugars'].tolist()

vitamin_d = food_data['Vitamin D'].to_list()
Calcium = food_data['Calcium'].to_list()
iron = food_data['Iron'].to_list()
Potassium = food_data['Potassium'].to_list()

In [19]:
m_male = gp.Model("TBELL_MODEL: EOM-LP")
m_male.Params.LogToConsole = 1 # Noisy output

foods = []
for i in range(len(food_data)):
    foods.append(m_male.addVar(obj=food_data['price'][i], vtype="C", name=food_data['item_name'][i]))


m_male.setObjective(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))), GRB.MINIMIZE)

constraints = {}
#### Constraints
### Nutrients 
# Macro
constraints['calories'] = m_male.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= 2400/3)
constraints['protein']=m_male.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3)
constraints['totalCarbohydratets']=m_male.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3)
constraints['dietaryFiber']=m_male.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3)

constraints['totfat'] = m_male.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3)

constraints['statFat'] = m_male.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3)
constraints['transFat'] = m_male.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) >= 0/3)
constraints['cholesterol'] = m_male.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data)))>=300/3)
constraints['sodium'] = m_male.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data)))>=2300/3)
constraints['sugars'] = m_male.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3)

# Micro
constraints["Potassium"]=m_male.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3)
constraints["iron"]=m_male.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3)
constraints["Calcium"]=m_male.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3)
constraints["vitamin_d"]=m_male.addConstr(gp.quicksum(vitamin_d[i]*foods[i] for i in range(len(food_data))) >= 15/3)


m_male.ModelSense = 1 # Minimization problem
m_male.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 14 rows, 69 columns and 703 nonzeros
Model fingerprint: 0x45420553
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [1e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+03]
Presolve removed 1 rows and 7 columns
Presolve time: 0.00s
Presolved: 13 rows, 62 columns, 685 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.023750e+02   0.000000e+00      0s
       3    3.9616453e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.961645320e+00


In [20]:
# Solution for TACOBELL MODEL 1: MILP
if m_male.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${round(m_male.objVal,2)}:')
 
    EOM1_df = output_to_table(m_male, foods, food_data)
EOM1_df



Optimal combination of foods for one meal costs $3.96:


,Menu,Servings,Nutrient,Values
0,Cost per Meal,$3.96,Calories,1267.6353043143297
1,Cheesy Bean and Rice Burrito,1.95,Protein (g),29.13962185413456
2,Fiesta Veie Burrito,0.26,Total Carbohydrate (g),163.0270127118644
3,Cheesy Toasted Breakfast Burrito Potato,0.88,Dietary Fiber (g),18.646635850025678
4,,,Total Fat (g),50.85114920390343
5,,,Saturated Fat (g),12.973203967642526
6,,,Trans Fat (g),0.0
7,,,Cholesterol (mg),99.99999999999999
8,,,Sodium (mg),2739.446102978942
9,,,Sugars (g),14.844311761684644


In [21]:
# Sensitivity Analysis on Model 1: LP
if m_male.status == gp.GRB.OPTIMAL:
    print("\nSensitivity Analysis")
    print(f"An additional gram of protein will cost {constraints['protein'].Pi}")
    print(f"An additional gram of carbs will cost {constraints['totalCarbohydratets'].Pi}")
    print(f"An additional gram of fiber will cost {constraints['dietaryFiber'].Pi}")
    print(f"An additional gram of total fat will cost {constraints['totfat'].Pi}")
    print(f"An additional gram of total fat will cost {constraints['totfat'].Pi}")

    print(f"An additional gram of saturated fat will cost {constraints['statFat'].Pi}")
    print(f"An additional gram of trans fat will cost {constraints['transFat'].Pi}")
    print(f"An additional gram of cholesterol will cost {round(constraints['cholesterol'].Pi, 4)}")
    print(f"An additional gram of sodium will cost {constraints['sodium'].Pi}")
    print(f"An additional gram of sugars will cost {constraints['sugars'].Pi}")

    print(f"An additional mg of Potassium will cost {round(constraints['Potassium'].Pi, 4)}")
    print(f"An additional mg of iron will cost {constraints['iron'].Pi}")
    print(f"An additional mg of calcium will cost {constraints['Calcium'].Pi}")
    print(f"An additional mcg of vitamin D will cost {round(constraints['vitamin_d'].Pi, 4)}")


Sensitivity Analysis
An additional gram of protein will cost 0.0
An additional gram of carbs will cost 0.0
An additional gram of fiber will cost 0.0
An additional gram of total fat will cost 0.0
An additional gram of total fat will cost 0.0
An additional gram of saturated fat will cost 0.0
An additional gram of trans fat will cost 0.0
An additional gram of cholesterol will cost 0.008
An additional gram of sodium will cost 0.0
An additional gram of sugars will cost 0.0
An additional mg of Potassium will cost 0.0027
An additional mg of iron will cost 0.0
An additional mg of calcium will cost 0.0
An additional mcg of vitamin D will cost 0.0285


The Pi attribute is also known as the dual value or shadow price and tells us how much the objective would change (total cost of meal) if the right-hand side (max nutrients required) of the constraints were increased by 1 unit.


## Changing 'Model 1' to an integer linear programming model

In [22]:
m_male_ILP = gp.Model("TBELL_MODEL: EOM-IP")
m_male_ILP.Params.LogToConsole = 1 # Noisy output


foods = []
for i in range(len(food_data)):
    foods.append(m_male_ILP.addVar(obj=food_data['price'][i], vtype="I", name=food_data['item_name'][i]))




m_male_ILP.setObjective(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))), GRB.MINIMIZE)

constraints_ILP = {}
#### Constraints
# Macro Nutrients  
constraints_ILP['calories'] = m_male_ILP.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= 2400/3)
constraints_ILP['protein']=m_male_ILP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3)
constraints_ILP['totalCarbohydratets']=m_male_ILP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3)
constraints_ILP['dietaryFiber']=m_male_ILP.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3)

constraints_ILP['totfat'] = m_male_ILP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3)

constraints_ILP['statFat'] = m_male_ILP.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3)
constraints_ILP['transFat'] = m_male_ILP.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) >= 0/3)
constraints_ILP['cholesterol'] = m_male_ILP.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data)))>=300/3) # not a min amount only max
constraints_ILP['sodium'] = m_male_ILP.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data)))>=2300/3)
constraints_ILP['sugars'] = m_male_ILP.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3)

# Micro Nutrients
constraints_ILP["Potassium"]=m_male_ILP.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3)
constraints_ILP["iron"]=m_male_ILP.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3)
constraints_ILP["Calcium"]=m_male_ILP.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3)
constraints_ILP["vitamin_d"]=m_male_ILP.addConstr(gp.quicksum(vitamin_d[i]*foods[i] for i in range(len(food_data))) >= 15/3)

m_male_ILP.ModelSense = 1 #Minimization problem

m_male_ILP.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 14 rows, 69 columns and 703 nonzeros
Model fingerprint: 0xd6e0cdcb
Variable types: 0 continuous, 69 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [1e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective 15.8200000
Presolve removed 5 rows and 44 columns
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 204 nonzeros
Found heuristic solution: objective 5.9900000
Variable types: 0 continuous, 25 integer (0 binary)

Root relaxation: objective 4.384237e+00, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.38424    0    2    5.99000


Cutting planes:
  MIR: 1

Explored 1 nodes (5 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 10 (of 10 available processors)

Solution count 3: 4.69 5.99 15.82 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.690000000000e+00, best bound 4.690000000000e+00, gap 0.0000%


In [23]:
if m_male_ILP.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${round(m_male_ILP.objVal,2)}:')


    # Creating the DataFrame
    EOM_IP1_df = output_to_table(m_male_ILP, foods, food_data)


EOM_IP1_df

Optimal combination of foods for one meal costs $4.69:


,Menu,Servings,Nutrient,Values
0,Cost per Meal,$4.69,Calories,1330.0
1,Cheesy Bean and Rice Burrito,1.0,Protein (g),32.0
2,Fiesta Veie Burrito,1.0,Total Carbohydrate (g),164.0
3,Cheesy Toasted Breakfast Burrito Potato,1.0,Dietary Fiber (g),19.0
4,,,Total Fat (g),58.0
5,,,Saturated Fat (g),15.5
6,,,Trans Fat (g),0.0
7,,,Cholesterol (mg),125.0
8,,,Sodium (mg),2710.0
9,,,Sugars (g),15.0


In [24]:
m_male2_ILP = gp.Model("TBELL_MODEL: EOM-IP2")
m_male2_ILP.Params.LogToConsole = 1 # Noisy output


foods = []
for i in range(len(food_data)):
    foods.append(m_male2_ILP.addVar(obj=food_data['price'][i], vtype="I", name=food_data['item_name'][i]))



m_male2_ILP.setObjective(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))), GRB.MINIMIZE)

constraints2_ILP = {}
#### Constraints
### Nutrients 
# Macro
constraints2_ILP['calories'] = m_male2_ILP.addConstr(gp.quicksum(calories[i]*foods[i] for i in range(len(food_data))) >= 2400/3)
constraints2_ILP['protein']=m_male2_ILP.addConstr(gp.quicksum(protein[i]*foods[i] for i in range(len(food_data))) >= 56/3)
constraints2_ILP['totalCarbohydratets']=m_male2_ILP.addConstr(gp.quicksum(totalCarbohydratets[i]*foods[i] for i in range(len(food_data))) >= 130/3)
constraints2_ILP['dietaryFiber']=m_male2_ILP.addConstr(gp.quicksum(dietaryFiber[i]*foods[i] for i in range(len(food_data))) >= 34/3)
constraints2_ILP['totfat'] = m_male2_ILP.addConstr(gp.quicksum(totfat[i]*foods[i] for i in range(len(food_data))) >= max_totFat/3)
constraints2_ILP['statFat'] = m_male2_ILP.addConstr(gp.quicksum(statFat[i]*foods[i] for i in range(len(food_data))) >= max_statFat/3)
constraints2_ILP['transFat'] = m_male2_ILP.addConstr(gp.quicksum(transFat[i]*foods[i] for i in range(len(food_data))) >= 0/3)
constraints2_ILP['cholesterol'] = m_male2_ILP.addConstr(gp.quicksum(cholesterol[i]*foods[i] for i in range(len(food_data)))>=300/3)
constraints2_ILP['sodium'] = m_male2_ILP.addConstr(gp.quicksum(sodium[i]*foods[i] for i in range(len(food_data)))>=2300/3)
constraints2_ILP['sugars'] = m_male2_ILP.addConstr(gp.quicksum(sugars[i]*foods[i] for i in range(len(food_data)))>=10/3)

# Micro
constraints2_ILP["Potassium"]=m_male2_ILP.addConstr(gp.quicksum(Potassium[i]*foods[i] for i in range(len(food_data))) >= 3400/3)
constraints2_ILP["iron"]=m_male2_ILP.addConstr(gp.quicksum(iron[i]*foods[i] for i in range(len(food_data))) >= 8/3)
constraints2_ILP["Calcium"]=m_male2_ILP.addConstr(gp.quicksum(Calcium[i]*foods[i] for i in range(len(food_data))) >= 1000/3)
constraints2_ILP["vitamin_d"]=m_male2_ILP.addConstr(gp.quicksum(vitamin_d[i]*foods[i] for i in range(len(food_data))) >= 15/3)

# Budget 
budget = 86.80/7   # mention where I found this number https://money.usnews.com/money/personal-finance/saving-and-budgeting/articles/how-much-should-i-spend-on-groceries

m_male2_ILP.addConstr(gp.quicksum(price[i]*foods[i] for i in range(len(food_data))) >= budget)

m_male2_ILP.ModelSense = 1 #Minimization problem
m_male2_ILP.optimize()



Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 15 rows, 69 columns and 772 nonzeros
Model fingerprint: 0x589ca51f
Variable types: 0 continuous, 69 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [1e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective 15.8200000
Presolve removed 4 rows and 22 columns
Presolve time: 0.00s
Presolved: 11 rows, 47 columns, 461 nonzeros
Found heuristic solution: objective 12.4400000
Variable types: 0 continuous, 47 integer (0 binary)

Root relaxation: objective 1.240000e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   12.40000    0    2   12.440

In [25]:
if m_male2_ILP.status == gp.GRB.OPTIMAL:
    print(f'Optimal combination of foods for one meal costs ${m_male2_ILP.objVal}:')

    EOM_IP2_df = output_to_table(m_male2_ILP, foods, food_data)


EOM_IP2_df


Optimal combination of foods for one meal costs $12.4:


,Menu,Servings,Nutrient,Values
0,Cost per Meal,$12.4,Calories,3240.0
1,Spicy Potato Soft Taco,4.0,Protein (g),76.0
2,Fiesta Veie Burrito,4.0,Total Carbohydrate (g),372.0
3,,,Dietary Fiber (g),44.0
4,,,Total Fat (g),160.0
5,,,Saturated Fat (g),44.0
6,,,Trans Fat (g),0.0
7,,,Cholesterol (mg),140.0
8,,,Sodium (mg),6000.0
9,,,Sugars (g),32.0


# Next todo
- Prevent next model from picking a mass quanity of the same product; adding constraint or an upper bound arg to decison variable.
- Adding the recommneded percentages of each nutrinal requriment as a percentage of daily caloric intake. The foluation for this is below:
    * $u_j$ = Maximun daily requirements of nutrient $j$, $j \in N$
    * $ m_j \leq\sum_{i \in F} n_{i,j} * x_i \leq u_j,  j \in N$

- Once the top two bulid points are complete this will allow me to start compairing the meals from the ILP model to the combos Taco Bell has to offer 
    - This model will also include a constraint that a food item must be picked from specific parts of the menu. i.e. drinks, tacos, burritos 